Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [34]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [35]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

#stock_symbol = "2222.SR"#Saudi Arabian Oil Company(ARAMCO)
stock_symbol ='BTC-USD'
#stock_symbol ='ETH-USD'

In [36]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.
end = dt.datetime.now()
start = dt.datetime(2020,11,8)
df = pdr.get_data_yahoo(stock_symbol, start, end)
df = df.iloc[:, :4]  #or use df[['Open','High','Low','Close']]
df =df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2020-11-07,15565.880859,15737.095703,14423.203125,14833.753906
2020-11-08,14833.753906,15637.320312,14744.110352,15479.567383
2020-11-09,15479.595703,15785.136719,14865.529297,15332.315430
2020-11-10,15332.350586,15450.329102,15124.959961,15290.902344
2020-11-11,15290.909180,15916.260742,15290.006836,15701.339844
...,...,...,...,...
2021-11-05,61460.078125,62541.468750,60844.609375,61125.675781
2021-11-06,61068.875000,61590.683594,60163.781250,61527.480469
2021-11-07,61554.921875,63326.988281,61432.488281,63326.988281


In [37]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".
df['Close_30_Rolling'] = df['Close'].rolling(window=30).mean()
df['Close_60_Rolling'] = df['Close'].rolling(window=60).mean()
df['Range']=df['High']-df['Low']
df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] =df['Open']-df['Open'].shift(1)


df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_30_Rolling,Close_60_Rolling,Range,Close_15_Rolling,Open_1_Change
Date,,,,,,,,,
2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,24235.881055,20909.673730,4216.402344,28044.810286,-833.908203
2021-01-06,34013.613281,36879.699219,33514.035156,36824.363281,24823.638802,21276.183887,3365.664062,28914.232552,2036.572266
2021-01-07,36833.875000,40180.367188,36491.191406,39371.042969,25525.302083,21674.375146,3689.175781,29989.545703,2820.261719
2021-01-08,39381.765625,41946.738281,36838.636719,40797.609375,26266.758529,22098.796712,5108.101562,31126.989714,2547.890625
2021-01-09,40788.640625,41436.351562,38980.875000,40254.546875,26999.743685,22514.857454,2455.476562,32166.306771,1406.875000
...,...,...,...,...,...,...,...,...,...
2021-11-05,61460.078125,62541.468750,60844.609375,61125.675781,60167.743359,53011.854948,1696.859375,61382.625000,-1481.726562
2021-11-06,61068.875000,61590.683594,60163.781250,61527.480469,60425.126563,53257.127474,1426.902344,61438.305990,-391.203125
2021-11-07,61554.921875,63326.988281,61432.488281,63326.988281,60737.097917,53544.387435,1894.500000,61567.197396,486.046875


In [38]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.
#r=df['Close'].shift(-1) - df['Close']
#print(r)
y = np.where(df['Close'].shift(-1) - df['Close'] >= 0, 1, -1)
#y       

In [39]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=index)#X_test &X_train ->data,y_train, y_test-->1&-1

In [40]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic =logistic.fit(X_train,y_train)
#logistic =logistic.fit(X_train,np.ravel(y_train.astype(int)))

In [41]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.00020750564194567197]
1,High,[-0.00012435618297688064]
2,Low,[-0.0001731273876225271]
3,Close,[0.0001618919312801939]
4,Close_30_Rolling,[2.4462688530385773e-05]
5,Close_60_Rolling,[3.0727126119421404e-05]
6,Range,[4.8771204645650636e-05]
7,Close_15_Rolling,[-0.0001266686446778075]
8,Open_1_Change,[-4.799940582687904e-05]


In [42]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities =logistic.predict_proba(X_test)# betwen 0 to 1
predictions = logistic.predict(X_test)#1 and -1 only
#print(probabilities)
#print(predictions)

In [43]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, predictions)
print(confusion_matrix)
#from sklearn.metrics import classification_report
#print(classification_report(y_test, predictions))

[[32 75]
 [28 81]]


In [44]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

accuracy=logistic.score(X_test,y_test)
print(accuracy)

0.5231481481481481


In [45]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".
from sklearn.model_selection import cross_val_score
cross_val =cross_val_score(logistic,X_test,y_test)
#cross_val2 =cross_val_score(logistic,X,y)
print(cross_val)
#print(cross_val2)

[0.47727273 0.55813953 0.48837209 0.48837209 0.39534884]


C:\Users\ali\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
